In [4]:
# Install the required libraries
#SAM
!pip install git+https://github.com/facebookresearch/segment-anything.git
#Transformers
!pip install -q git+https://github.com/huggingface/transformers.git
#Datasets to prepare data and monai if you want to use special loss functions
!pip install datasets
!pip install -q monai
#Patchify to divide large images into smaller patches for training. (Not necessary for smaller images)
!pip install patchify

!pip install matplotlib

!pip install tifffile

!pip install scipy


  Cloning https://github.com/facebookresearch/segment-anything.git to /private/var/folders/80/kgpm907s6db4tz7cvmwtd6l80000gn/T/pip-req-build-i8eutbnl
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /private/var/folders/80/kgpm907s6db4tz7cvmwtd6l80000gn/T/pip-req-build-i8eutbnl
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36587 sha256=e8d69d6e145ac0d9de640893a55e7dda8a3b7fb1c6c3da3835a88979308badcb
  Stored in directory: /private/var/folders/80/kgpm907s6db4tz7cvmwtd6l80000gn/T/pip-ephem-wheel-cache-37h8s26e/wheels/15/d7/bd/05f5f23b7dcbe70cbc6783b06f12143b0cf1a5da5c7b52dcc5
Successfully built segment_anything
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [7]:
#Numpy for handling data in vectors, matplotlib to graph
import numpy as np
import matplotlib.pyplot as plt
#Library to work with TIFFs
import tifffile

#Importing OS for 
import os
#We are going to work with large images and patchify helps with dividing it up
import patchify 
import scipy
from scipy import ndimage
#Random imported for sampling
import random
import cv2

In [10]:
def read_image_and_mask(image_path, mask_path):
    """
    Reads an image and a mask from the specified paths and returns them as NumPy arrays.
    
    Args:
        image_path (str): Path to the image file.
        mask_path (str): Path to the mask file.
    
    Returns:
        tuple: A tuple containing the image array and the mask array.
    """
    # Read the image using OpenCV
    image = cv2.imread(image_path)

    # Read the mask image using OpenCV
    mask_image = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Convert the images to NumPy arrays
    image_array = np.array(image)
    mask_array = np.array(mask_image)

    return image_array, mask_array

# Example usage:
image_array, mask_array = read_image_and_mask('/Users/adikrish/Desktop/Adam.jpg', '/Users/adikrish/Desktop/Eve.png')

In [ ]:
#creates a step size and a patches
patch_size = 256 
step = 256